# Week 2
for this weeks assignment i chose to do text clustering. for this assignment i will pick out my own dataset and perform text clustering on it.


# Imports

In [1]:
import pandas as pd
import PyPDF2
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.feature_extraction import text
from nltk.corpus import stopwords
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
import re
import string
import glob
import pandas as pd
from pathlib import Path

In [2]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     /homes/mkwierenga/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /homes/mkwierenga/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /homes/mkwierenga/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /homes/mkwierenga/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

# Step 1: load the PDF file


In [3]:
reader = PyPDF2.PdfReader('data/machinelearningbook.pdf')

## step 2: extract the lines to dataframe

In [4]:
data = []
chapter = 0
for i in range(14, 369):
    page = reader.pages[i].extract_text().replace('\n', ' ').split('.')
    for line in page:
        if 'CHAPTER' in line:
            chapter +=1
        line = line.lower()
        line = re.sub('\[.*?\]', ' ', line)
        line = re.sub('[%s]' % re.escape(string.punctuation), ' ', line)
        line = re.sub('\w*\d\w*', ' ', line)
        line = re.sub('�', ' ', line)
        data.append([chapter, line, i])
df = pd.DataFrame(data, columns=['chapter', 'line', 'page_number'])

In [5]:
# Noun extract and lemmatize function
def nouns(text):
    '''Given a string of text, tokenize the text 
    and pull out only the nouns.'''
    # create mask to isolate words that are nouns
    is_noun = lambda pos: pos[:2] == 'NN'
    # store function to split string of words 
    # into a list of words (tokens)
    tokenized = word_tokenize(text)
    # store function to lemmatize each word
    wordnet_lemmatizer = WordNetLemmatizer()
    # use list comprehension to lemmatize all words 
    # and create a list of all nouns
    all_nouns = [wordnet_lemmatizer.lemmatize(word) \
    for (word, pos) in pos_tag(tokenized) if is_noun(pos)] 
    
    #return string of joined list of nouns
    return ' '.join(all_nouns)



In [6]:
data_nouns = pd.DataFrame(df["line"].apply(nouns))
# Visually Inspect
data_nouns

,line
0,chapter introduction machine learning knowledg...
1,research field intersection statistic intellig...
2,application machine method year life
3,recommendation movie food order product online...
4,website facebook amazon netflix part site mach...
...,...
7520,development text processing year scope book re...
7521,use vector representation word vector word rep...
7522,paper “ representation word phrase composi‐ ti...
7523,introduction subject


In [31]:
merged_df = pd.merge(df['chapter'], data_nouns, left_index=True, right_index=True)
merged_df

,chapter,line
0,1,chapter introduction machine learning knowledg...
1,1,research field intersection statistic intellig...
2,1,application machine method year life
3,1,recommendation movie food order product online...
4,1,website facebook amazon netflix part site mach...
...,...,...
7520,7,development text processing year scope book re...
7521,7,use vector representation word vector word rep...
7522,7,paper “ representation word phrase composi‐ ti...
7523,7,introduction subject


In [110]:
def to_dtm_matrix(data_nouns):
    # Create a document-term matrix with only nouns
    # Store TF-IDF Vectorizer
    tv_noun = TfidfVectorizer(stop_words=list(text.ENGLISH_STOP_WORDS), ngram_range = (1,1), max_df = .8, min_df = .01)
    # Fit and Transform speech noun text to a TF-IDF Doc-Term Matrix
    data_tv_noun = tv_noun.fit_transform(data_nouns.line)
    # Create data-frame of Doc-Term Matrix with nouns as column names
    data_dtm_noun = pd.DataFrame(data_tv_noun.toarray(), columns=tv_noun.get_feature_names_out())
#     data_dtm_noun.index = df.index
    # Visually inspect Document Term Matrix
    return tv_noun, data_dtm_noun

In [114]:
def display_topics(model, feature_names, num_top_words, topic_names=None):
    '''Given an NMF model, feature_names, and number of top words, print 
       topic number and its top feature names, up to specified number of top words.'''
    # iterate through topics in topic-term matrix, 'H' aka
    # model.components_
    for ix, topic in enumerate(model.components_):
        #print topic, topic number, and top words
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i] \
             for i in topic.argsort()[:-num_top_words - 1:-1]]))




In [128]:
data_dtm_noun = [to_dtm_matrix(merged_df[merged_df['chapter'] == 1][['line']]), 
                 to_dtm_matrix(merged_df[merged_df['chapter'] == 2][['line']]),
                 to_dtm_matrix(merged_df[merged_df['chapter'] == 3][['line']]),
                 to_dtm_matrix(merged_df[merged_df['chapter'] == 4][['line']]), 
                 to_dtm_matrix(merged_df[merged_df['chapter'] == 5][['line']]),
                 to_dtm_matrix(merged_df[merged_df['chapter'] == 6][['line']]),
                 to_dtm_matrix(merged_df[merged_df['chapter'] == 7][['line']])]


In [135]:
chapter = 1
for data in data_dtm_noun: 
    nmf_model = NMF(5)
    # Learn an NMF model for given Document Term Matrix 'V' 
    # Extract the document-topic matrix 'W'
    doc_topic = nmf_model.fit_transform(data[1])
    # Extract top words from the topic-term matrix 'H' 
    doc_topic
    print(f'chapter: {chapter}')
    display_topics(nmf_model, data[0].get_feature_names_out(), 10)
    print('\n')
    chapter +=1

chapter: 1

Topic  0
data, array, point, feature, label, training, measurement, flower, iris, output

Topic  1
scikit, version, learn, import, matplotlib, panda, scipy, ipython, algorithm, tool

Topic  2
machine, model, learning, application, chapter, dataset, problem, specie, introduction, algorithm

Topic  3
python, code, package, notebook, library, jupyter, output, tool, scipy, matplotlib

Topic  4
format, test, train, set, score, print, prediction, shape, knn, model


chapter: 2

Topic  0
model, data, tree, point, parameter, decision, class, regression, prediction, algorithm

Topic  1
test, score, train, accuracy, set, training, print, fit, ax, prediction

Topic  2
feature, plt, tree, ax, ylabel, dataset, class, input, cancer, chapter

Topic  3
format, gbrt, mlp, np, cancer, neighbor, train, ridge, class, state

Topic  4
figure, plot, mglearn, dataset, ax, decision, layer, boundary, neighbor, result


chapter: 3

Topic  0
cluster, point, center, sample, number, mean, min, label, km